# Crypto Quote Analysis Report

In [22]:
import pandas as pd
from google.cloud import bigquery
import plotly.express as px

# Adjust pandas display options
pd.options.plotting.backend = 'plotly'

PROJECT_ID = "unbiased-reporting"
DATASET_ID = "articles"
TABLE_ID = "crypto_quotes"

client = bigquery.Client(project=PROJECT_ID)

## 1. Load Data from BigQuery

In [23]:
query = f"""SELECT * FROM `{PROJECT_ID}.{DATASET_ID}.{TABLE_ID}`"""

df = client.query(query).to_dataframe()

print(f"Loaded {len(df)} rows from BigQuery.")
df.head()

/Users/nimaakram/Library/Caches/pypoetry/virtualenvs/crypto-prices-oQTLIRdB-py3.11/lib/python3.11/site-packages/google/cloud/bigquery/table.py:1994: UserWarning:

BigQuery Storage module not found, fetch data with the REST endpoint instead.



Loaded 2352 rows from BigQuery.


,Timestamp,Amount,FiatCurrency,CryptoCurrency,Region,PaymentMethod,Provider,Rank,AmountOut,ExchangeRate,MarketRate,ExpectedAmountOut,Spread,SpreadPercentage,NetworkFee,ProviderFee,ExtraFee,TotalExplicitFee,TotalFeeIncludingSpread,TotalFeePercentage
0,2025-08-31 23:55:18.385371+00:00,100.0,USD,USDT (Ethereum),US-VA,venmo,Moonpay,1,92.910000,1.014315,NaN,0.0,0.0,0.0,0.58,5.18,0.0,5.76,5.76,5.76
1,2025-08-31 23:55:20.273741+00:00,100.0,USD,USDT (Ethereum),US-VA,debit-credit-card,Moonpay,1,98.720000,0.970624,NaN,0.0,0.0,0.0,0.58,3.60,0.0,4.18,4.18,4.18
2,2025-08-31 23:55:20.273765+00:00,100.0,USD,USDT (Ethereum),US-VA,debit-credit-card,Ramp Network,2,95.497759,1.000338,NaN,0.0,0.0,0.0,0.18,4.29,0.0,4.47,4.47,4.47
3,2025-08-31 23:55:20.273770+00:00,100.0,USD,USDT (Ethereum),US-VA,debit-credit-card,Transak,3,94.290000,1.017605,NaN,0.0,0.0,0.0,0.05,4.00,0.0,4.05,4.05,4.05
4,2025-08-31 23:55:20.273775+00:00,100.0,USD,USDT (Ethereum),US-VA,debit-credit-card,Mercuryo,4,95.315930,1.000043,NaN,0.0,0.0,0.0,0.08,4.60,0.0,4.68,4.68,4.68


# Create Price Bins

In [24]:
print("Creating price bins...")
bins = [0, 500, 5000, 30001]
labels = ['Low (0-499)', 'Medium (500-4999)', 'High (5k+)']
df['PriceBin'] = pd.cut(df['Amount'], bins=bins, labels=labels, right=False)

Creating price bins...


# --- 2. Analysis: Average Rank by Price Bin ---

In [25]:
print("Generating individual Rank Analysis charts...")
unique_combinations = df[['CryptoCurrency', 'Region']].drop_duplicates()

for index, row in unique_combinations.iterrows():
    crypto = row['CryptoCurrency']
    region = row['Region']
    
    subset_df = df[(df['CryptoCurrency'] == crypto) & (df['Region'] == region)]
    
    if not subset_df.empty:
        avg_rank_df = subset_df.groupby(['PriceBin', 'Provider'])['Rank'].mean().reset_index()
        
        fig_rank = px.bar(avg_rank_df, 
                          x='Provider', y='Rank', 
                          color='Provider',
                          facet_col='PriceBin',
                          category_orders={'PriceBin': labels},
                          title=f'Average Provider Rank in {region} for {crypto}',
                          labels={'Rank': 'Average Rank (Lower is Better)'})
        
        fig_rank.update_yaxes(autorange="reversed")
        
        # Sanitize filename
        safe_crypto = "".join(c for c in crypto if c.isalnum() or c in (' ', '_')).rstrip()
        fig_rank.show()

Generating individual Rank Analysis charts...


/var/folders/8d/7rw9gkq50sz757_5bvsbvt600000gn/T/ipykernel_49887/1404418029.py:11: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



/var/folders/8d/7rw9gkq50sz757_5bvsbvt600000gn/T/ipykernel_49887/1404418029.py:11: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



/var/folders/8d/7rw9gkq50sz757_5bvsbvt600000gn/T/ipykernel_49887/1404418029.py:11: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



/var/folders/8d/7rw9gkq50sz757_5bvsbvt600000gn/T/ipykernel_49887/1404418029.py:11: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



/var/folders/8d/7rw9gkq50sz757_5bvsbvt600000gn/T/ipykernel_49887/1404418029.py:11: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



/var/folders/8d/7rw9gkq50sz757_5bvsbvt600000gn/T/ipykernel_49887/1404418029.py:11: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



/var/folders/8d/7rw9gkq50sz757_5bvsbvt600000gn/T/ipykernel_49887/1404418029.py:11: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



/var/folders/8d/7rw9gkq50sz757_5bvsbvt600000gn/T/ipykernel_49887/1404418029.py:11: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



/var/folders/8d/7rw9gkq50sz757_5bvsbvt600000gn/T/ipykernel_49887/1404418029.py:11: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



## 3. Analysis: Total Fee % vs. Amount

In [26]:
print("Generating individual Fee Analysis charts...")
for index, row in unique_combinations.iterrows():
    crypto = row['CryptoCurrency']
    region = row['Region']
    
    subset_df = df[(df['CryptoCurrency'] == crypto) & (df['Region'] == region)]
    
    if not subset_df.empty:
        avg_fee_df = subset_df.groupby(['PriceBin', 'Provider'])['TotalFeePercentage'].mean().reset_index()
        
        fig_fee = px.bar(avg_fee_df, 
                         x='Provider', y='TotalFeePercentage', 
                         color='Provider', 
                         facet_col='PriceBin',
                         category_orders={'PriceBin': labels},
                         title=f'Average Total Fee % in {region} for {crypto}',
                         labels={'TotalFeePercentage': 'Average Fee (%)'})
        
        # Sanitize filename
        safe_crypto = "".join(c for c in crypto if c.isalnum() or c in (' ', '_')).rstrip()
        filename = f"fee_charts/Fee_{safe_crypto}_{region}.html"
        fig_fee.show()

Generating individual Fee Analysis charts...


/var/folders/8d/7rw9gkq50sz757_5bvsbvt600000gn/T/ipykernel_49887/962148720.py:9: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



/var/folders/8d/7rw9gkq50sz757_5bvsbvt600000gn/T/ipykernel_49887/962148720.py:9: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



/var/folders/8d/7rw9gkq50sz757_5bvsbvt600000gn/T/ipykernel_49887/962148720.py:9: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



/var/folders/8d/7rw9gkq50sz757_5bvsbvt600000gn/T/ipykernel_49887/962148720.py:9: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



/var/folders/8d/7rw9gkq50sz757_5bvsbvt600000gn/T/ipykernel_49887/962148720.py:9: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



/var/folders/8d/7rw9gkq50sz757_5bvsbvt600000gn/T/ipykernel_49887/962148720.py:9: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



/var/folders/8d/7rw9gkq50sz757_5bvsbvt600000gn/T/ipykernel_49887/962148720.py:9: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



/var/folders/8d/7rw9gkq50sz757_5bvsbvt600000gn/T/ipykernel_49887/962148720.py:9: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



/var/folders/8d/7rw9gkq50sz757_5bvsbvt600000gn/T/ipykernel_49887/962148720.py:9: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



# --- 4. Analysis: Rank vs. Fee by Price Bin ---

In [32]:
print("Generating individual Rank vs. Fee Analysis charts...")
for index, row in unique_combinations.iterrows():
    crypto = row['CryptoCurrency']
    region = row['Region']
    
    subset_df = df[(df['CryptoCurrency'] == crypto) & (df['Region'] == region)]
    
    if not subset_df.empty:
        avg_df = subset_df.groupby(['PriceBin', 'Provider', 'PaymentMethod']).agg({'Rank': 'mean', 'TotalFeePercentage': 'mean'}).reset_index()
        
        fig_scatter = px.scatter(avg_df, 
                                 x='Rank', y='TotalFeePercentage', 
                                 color='Provider',
                                 symbol='PaymentMethod',
                                 facet_col='PriceBin',
                                 category_orders={'PriceBin': labels},
                                 title=f'Average Rank vs. Fee % in {region} for {crypto}',
                                 labels={'Rank': 'Average Rank', 'TotalFeePercentage': 'Average Fee (%)'})
        
        # Sanitize filename
        safe_crypto = "".join(c for c in crypto if c.isalnum()or c in (' ', '_')).rstrip()
        fig_scatter.show()

print("Analysis complete.")

Generating individual Rank vs. Fee Analysis charts...


/var/folders/8d/7rw9gkq50sz757_5bvsbvt600000gn/T/ipykernel_49887/47067503.py:9: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



/var/folders/8d/7rw9gkq50sz757_5bvsbvt600000gn/T/ipykernel_49887/47067503.py:9: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



/var/folders/8d/7rw9gkq50sz757_5bvsbvt600000gn/T/ipykernel_49887/47067503.py:9: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



/var/folders/8d/7rw9gkq50sz757_5bvsbvt600000gn/T/ipykernel_49887/47067503.py:9: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



...

/var/folders/8d/7rw9gkq50sz757_5bvsbvt600000gn/T/ipykernel_49887/47067503.py:9: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



/var/folders/8d/7rw9gkq50sz757_5bvsbvt600000gn/T/ipykernel_49887/47067503.py:9: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



/var/folders/8d/7rw9gkq50sz757_5bvsbvt600000gn/T/ipykernel_49887/47067503.py:9: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



/var/folders/8d/7rw9gkq50sz757_5bvsbvt600000gn/T/ipykernel_49887/47067503.py:9: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



Analysis complete.
